---
title: "A Replication of Karlan and List (2007)"
author: "Yihang Huang"
date: today
callout-appearance: minimal # this hides the blue "i" icon on .callout-notes
---


## Introduction

Dean Karlan at Yale and John List at the University of Chicago conducted a large-scale **natural field experiment** to test the effectiveness of different fundraising strategies, specifically focusing on the role of **matching grants** in charitable giving. The study was published in the _American Economic Review_ in 2007.

To conduct the experiment, they collaborated with a politically liberal nonprofit organization in the United States. They sent direct mail solicitations to **50,083 previous donors**, randomly assigning them to one of several treatment groups:

- A **control group** that received a standard fundraising letter with no mention of a matching gift.
- Several **treatment groups**, which received letters mentioning a **matching grant** from a “concerned fellow member,” with variations in:
  - The **match ratio**: $1:$1, $2:$1, or $3:$1.
  - The **maximum matching grant amount**: $25,000, $50,000, or $100,000.
  - The **suggested donation amount**: three levels relative to the recipient's historical prior contribution.

Each letter was otherwise identical in content and conformed to the nonprofit’s usual style and language. By using real donation requests and real monetary stakes, this experiment goes beyond hypothetical survey-based approaches and offers insight into **actual donor behavior** in a natural setting.

This project seeks to replicate their findings using the publicly available data, focusing on the following main outcomes:
- The **response rate** (proportion of individuals who donated).
- The **revenue per solicitation** (total amount raised divided by number of solicitations).
- The effect of **different match ratios** on both response rate and revenue.
- Whether **larger match ratios** lead to proportionally greater donations.

The original article and supporting dataset are available through the [AEA website](https://www.aeaweb.org/articles?id=10.1257/aer.97.5.1774) and from Innovations for Poverty Action via [Harvard's Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/27853&version=4.2).


This project seeks to replicate their results.


## Data

### Description

The dataset consists of one row per previous donor who received a fundraising letter. Key variables include whether the individual received a treatment letter or not ('treatment'), the size of the matching ratio ('ratio'), and donation outcomes ('gave', 'amount'). It also includes demographic and behavioral variables such as prior donation frequency (freq), recency ('mrm2'), and location-level characteristics (e.g., 'red0', 'pwhite').

:::: {.callout-note collapse="true"}
### Variable Definitions

| Variable             | Description                                                         |
|----------------------|---------------------------------------------------------------------|
| `treatment`          | Treatment                                                           |
| `control`            | Control                                                             |
| `ratio`              | Match ratio                                                         |
| `ratio2`             | 2:1 match ratio                                                     |
| `ratio3`             | 3:1 match ratio                                                     |
| `size`               | Match threshold                                                     |
| `size25`             | \$25,000 match threshold                                            |
| `size50`             | \$50,000 match threshold                                            |
| `size100`            | \$100,000 match threshold                                           |
| `sizeno`             | Unstated match threshold                                            |
| `ask`                | Suggested donation amount                                           |
| `askd1`              | Suggested donation was highest previous contribution                |
| `askd2`              | Suggested donation was 1.25 x highest previous contribution         |
| `askd3`              | Suggested donation was 1.50 x highest previous contribution         |
| `ask1`               | Highest previous contribution (for suggestion)                      |
| `ask2`               | 1.25 x highest previous contribution (for suggestion)               |
| `ask3`               | 1.50 x highest previous contribution (for suggestion)               |
| `amount`             | Dollars given                                                       |
| `gave`               | Gave anything                                                       |
| `amountchange`       | Change in amount given                                              |
| `hpa`                | Highest previous contribution                                       |
| `ltmedmra`           | Small prior donor: last gift was less than median \$35              |
| `freq`               | Number of prior donations                                           |
| `years`              | Number of years since initial donation                              |
| `year5`              | At least 5 years since initial donation                             |
| `mrm2`               | Number of months since last donation                                |
| `dormant`            | Already donated in 2005                                             |
| `female`             | Female                                                              |
| `couple`             | Couple                                                              |
| `state50one`         | State tag: 1 for one observation of each of 50 states; 0 otherwise  |
| `nonlit`             | Nonlitigation                                                       |
| `cases`              | Court cases from state in 2004-5 in which organization was involved |
| `statecnt`           | Percent of sample from state                                        |
| `stateresponse`      | Proportion of sample from the state who gave                        |
| `stateresponset`     | Proportion of treated sample from the state who gave                |
| `stateresponsec`     | Proportion of control sample from the state who gave                |
| `stateresponsetminc` | stateresponset - stateresponsec                                     |
| `perbush`            | State vote share for Bush                                           |
| `close25`            | State vote share for Bush between 47.5% and 52.5%                   |
| `red0`               | Red state                                                           |
| `blue0`              | Blue state                                                          |
| `redcty`             | Red county                                                          |
| `bluecty`            | Blue county                                                         |
| `pwhite`             | Proportion white within zip code                                    |
| `pblack`             | Proportion black within zip code                                    |
| `page18_39`          | Proportion age 18-39 within zip code                                |
| `ave_hh_sz`          | Average household size within zip code                              |
| `median_hhincome`    | Median household income within zip code                             |
| `powner`             | Proportion house owner within zip code                              |
| `psch_atlstba`       | Proportion who finished college within zip code                     |
| `pop_propurban`      | Proportion of population urban within zip code                      |

::::


### Balance Test 

As an ad hoc test of the randomization mechanism, I provide a series of tests that compare aspects of the treatment and control groups to assess whether they are statistically significantly different from one another.

To test the integrity of the randomization, I compare pre-treatment characteristics such as 'mrm2', 'female', and 'freq' between the treatment and control groups.

In [ ]:
from scipy.stats import ttest_ind
import statsmodels.formula.api as smf
# Subset the variable to compare
treat = df[df['treatment'] == 1]['mrm2']
control = df[df['treatment'] == 0]['mrm2']
# T-test
t_stat, p_val = ttest_ind(treat, control, equal_var=False)
print(f"T-test: t = {t_stat:.3f}, p = {p_val:.3f}")

Using both Welch's t-tests and bivariate regressions, I confirm that none of the selected covariates are statistically significantly different at the 5% level. These results match Table 1 of Karlan and List (2007), indicating that the treatment was successfully randomized and there are no meaningful pre-treatment differences.

## Experimental Results

### Charitable Contribution Made

First, I analyze whether matched donations lead to an increased response rate of making a donation. 

A barplot comparing the response rates between groups shows that the treatment group had a higher donation rate than the control group.

A Welch's t-test confirms this difference is statistically significant (p < 0.01), and a bivariate linear regression of 'gave ~ treatment' returns a positive and significant coefficient. This matches the finding in Table 2a, Panel A.

A probit regression, as shown in Table 3 Column 1 of the original study, confirms that being in the treatment group significantly increases the probability of donating. The coefficient on treatment is around 0.325, and statistically significant at the 1% level.

Interpretation: These results show that offering a matching grant increases the likelihood of donating. People respond not just to financial incentives, but also to social and framing cues that increase perceived impact.


### Differences between Match Rates

Next, I assess the effectiveness of different sizes of matched donations on the response rate.

Using t-tests, I find no statistically significant difference in response rates between the 1:1 and 2:1 match groups, nor between 2:1 and 3:1. This supports the authors' observation that "increasing the match ratio above 1:1 does not further increase the probability that an individual gives."

In a regression of 'gave ~ ratio1 + ratio2 + ratio3', the coefficients are not statistically different from each other. Manually computing response rate differences from the data confirms this: the increase from 1:1 to 2:1 is about 0.0003, and from 2:1 to 3:1 is about -0.0002.

Conclusion: Higher match ratios do not significantly increase the probability of giving, confirming that simply including a match matters more than its size.


### Size of Charitable Contribution

In this subsection, I analyze the effect of the size of matched donation on the size of the charitable contribution.

A regression of 'amount ~ treatment' shows a small, positive, and statistically significant effect of treatment on donation amount. However, when restricting to individuals who actually donated ('gave == 1'), the coefficient becomes smaller and statistically insignificant.

Histograms of donation amounts for the control and treatment groups (among givers only) show similar distributions, with nearly identical sample means.

Interpretation: The treatment does not affect the size of donations once someone has decided to give. The causal impact lies in encouraging whether someone donates, not how much they give.


## Simulation Experiment

As a reminder of how the t-statistic "works," in this section I use simulation to demonstrate the Law of Large Numbers and the Central Limit Theorem.

Suppose the true distribution of respondents who do not get a charitable donation match is Bernoulli with probability p=0.018 that a donation is made. 

Further suppose that the true distribution of respondents who do get a charitable donation match of any size  is Bernoulli with probability p=0.022 that a donation is made.

### Law of Large Numbers

I simulated 100,000 draws from the control distribution and 10,000 from the treatment distribution. I then computed 10,000 differences and plotted their cumulative average.

The plot shows the cumulative average stabilizing around the true difference (0.004), consistent with the Law of Large Numbers: as sample size increases, the sample mean converges to the true mean.


### Central Limit Theorem

I ran simulations with sample sizes of 50, 200, 500, and 1000. For each size, I drew 1000 samples and computed the difference in means. The resulting histograms become increasingly normal and concentrated as sample size increases.

In small samples, the distribution is wide and skewed. At larger sizes, it becomes bell-shaped and centered near the true difference. Zero clearly moves from being in the middle (in small samples) to being in the tail (in larger samples).

This demonstrates the Central Limit Theorem: the sampling distribution of the mean difference becomes approximately normal as sample size increases.
